In [2]:
import numpy as np
import pandas as pd

In [3]:
vendor_review_df=pd.read_json("../r2Data/yelp/yelp_academic_dataset_review.json",lines=True,nrows=400000)

In [4]:
vendor_business_info=pd.read_json("../r2Data/yelp/yelp_academic_dataset_business.json",lines=True)